# Hands on : introduction to NN on HEP dataset

### Many thanks to _Fernando Torales Acosta, David Rousseau, Yann Coadou_, and _Aishik Gosh_

## Import Packages

[This should look familiar!]

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot as ur

import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from IPython.display import display, HTML
%matplotlib inline
import time
pd.set_option('display.max_columns', None) # to see all columns of df.head()
np.random.seed(31415) # set the random seed for the reproducibility

In [ ]:
print(tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Load events

data was created from ATLAS Open Data

In [ ]:
filename=("dataWW_d1.root")
file = ur.open(filename)
print(file.classnames())

In [ ]:
tree = file["tree_event"]
print(tree.keys())

In [ ]:
print(type(tree))
tree.show()
dfall = tree.arrays(library="pd")

[uproot](https://indico.cern.ch/event/686641/contributions/2894906/attachments/1606247/2548596/pivarski-uproot.pdf)

In [ ]:
#shuffle the events [already done but just to be safe]
dfall = dfall.sample(frac=1).reset_index(drop=True)
print ("File loaded with ",dfall.shape[0], " events ")

#### At this point, you should see "File Loaded with 600000 events". If not, the data file could not be accessed. No point going further!

# Examine Pandas Dataset

In [ ]:
#dump list of feature
dfall.columns

In [ ]:
#examine first few events
dfall.head()

In [ ]:
#take a look at feature distribution
dfall.describe()
#dfall.head(5)

In [ ]:
label_weights = (dfall[dfall.label==0].mcWeight.sum(), dfall[dfall.label==1].mcWeight.sum() ) 
print("total label weights",label_weights)

label_nevents = (dfall[dfall.label==0].shape[0], dfall[dfall.label==1].shape[0] )
print ("total class number of events",label_nevents)

## Event Selection

This notebook essentially tries to classify events containing a Higgs Boson.

The simulation includes top-quark-pair production, single-top production, production of weak bosons in association with jets (W+jets, Z+jets), production of a pair of bosons (diboson WW, WZ, ZZ) and __SM Higgs__ production.

We will only keep events with exactly two leptons dfall.lep_n==2

In [ ]:
print (dfall.shape)

# Also only keep events with positive weight. This is in principle wrong. 
#Many Data Science tools break given a negative weight.

fulldata=dfall[(dfall.lep_n==2) & (dfall.mcWeight>0 )] # only keep events with exactly two leptons 
print (fulldata.shape)

In [ ]:
#hide label and weights in separate vectors
#they are not real features

#WARNING : there should be no selection nor shuffling later on !
target = fulldata["label"]
del fulldata["label"]

#hide weight in separate vector
weights = fulldata["mcWeight"]
del fulldata["mcWeight"]
fulldata.shape

___


# Try not to change the cells above $\uparrow$
...and return to this cell (or rerun the whole notebook) after changing things below.

___

### For simplicity, we'll only use some features on the first pass

In [ ]:
data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_phi_0', 'lep_phi_1'])
print (data.shape)
data.head()

### Feature engineering (Two variations)

1. See if using more features improves model performance

In [ ]:
more_features = False
if (more_features):
    data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_eta_0', 'lep_eta_1', 'lep_phi_0', 'lep_phi_1','jet_n','jet_pt_0',
       'jet_pt_1', 'jet_eta_0', 'jet_eta_1', 'jet_phi_0', 'jet_phi_1'])

2. Engineer our own feature, $\Delta\varphi_l$

In [ ]:
use_deltaphi = False
if use_deltaphi: 
    data["lep_deltaphi"]=np.abs(np.mod(data.lep_phi_1-data.lep_phi_0+3*np.pi,2*np.pi)-np.pi)

    print (data.shape)
    display(data.head())

In [ ]:
plt.figure()

ax=data[target==0].hist(weights=weights[target==0],figsize=(15,12),color='b',alpha=0.5,density=True)
ax=ax.flatten()[:data.shape[1]] # to avoid error if holes in the grid of plots (like if 7 or 8 features)
data[target==1].hist(weights=weights[target==1],figsize=(15,12),color='r',alpha=0.5,density=True,ax=ax)

plt.show()

# Transform the features

### Split the Data into Training and Test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
train_size = 0.1 # fraction of sample used for training
# Try changing fraction to increase or decrease sample

X_train, X_test, y_train, y_test, weights_train, weights_test = \
    train_test_split(data, target, weights, train_size=train_size)

y_train, y_test, weights_train, weights_test = \
    y_train.reset_index(drop=True),y_test.reset_index(drop=True), \
    weights_train.reset_index(drop=True), weights_test.reset_index(drop=True)

print ("Xtrain Shape: ",X_train.shape)
print ("ytrain Shape: ",y_train.shape)
print ("Training Weights: ",weights_train.shape,"\n")
print ("Xtest Shape: ",X_test.shape)
print ("ytest Shape: ",y_test.shape)
print ("Test Weights: ",weights_test.shape)

### Doing an extra data split. Test _and_ Validation

In [ ]:
X_test, X_val, y_test, y_val, weights_test, weights_val, = \
    train_test_split(X_test, y_test, weights_test, train_size=0.5, shuffle=False)

- __Training Dataset:__ The sample of data used to fit the model.
- __Validation Dataset:__ The sample used to provide an unbiased evaluation of a model fit on the training dataset while tuning  hyperparameters.
- __Test Dataset:__ The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.

## Standardize the Data

**Scale to Mean of 0 and Variance of 1.0:**   $\ \ \ \ (x-\mu)/\sigma$

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test) #applies the transformation calculated the line above

### Adjust the Test and Train Signal/Background Weights
Train on equal amount of Signal and Background, Test on 'natural' ratio

In [ ]:
class_weights_train = (weights_train[y_train == 0].sum(), weights_train[y_train == 1].sum())
print ("class_weights_train:",class_weights_train)
for i in range(len(class_weights_train)):
    weights_train[y_train == i] *= max(class_weights_train)/ class_weights_train[i] #equalize number of background and signal event
    weights_test[y_test == i] *= 1/(1-train_size) #increase test weight to compensate for sampling
    
print ("Train : total weight sig", weights_train[y_train == 1].sum())
print ("Train : total weight bkg", weights_train[y_train == 0].sum())
print ("Test : total weight sig", weights_test[y_test == 1].sum())
print ("Test : total weight bkg", weights_test[y_test == 0].sum())

# Trying NN

## NN

In [ ]:
#Quickly take a look at weights
print(X_train.shape)
print(class_weights_train)
print(weights_train)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)), # input layer
    tf.keras.layers.Dense(128, activation='relu'), # 1st hiddden layer
    tf.keras.layers.Dense(128, activation='relu'), # 2nd hiddden layer
    tf.keras.layers.Dense(1,activation="sigmoid") # output layer
])
model.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
starting_time = time.time( )
the_fit = model.fit(X_train, y_train.values, epochs=10,
                     sample_weight=weights_train)
#not using validation dataset here; keras does not support val weights
#Solution: Define your own loss function that takes train and val loss! 

training_time = time.time( ) - starting_time
print("Training time:",training_time)

In [ ]:
plt.plot(the_fit.history['loss'],label="training loss")
plt.legend(fontsize=15)

### Use the model to make predicions!
Evaluate the model based on predictions made with X_test $\rightarrow$ y_test

In [ ]:
y_pred_test = model.predict(X_test).ravel()
y_pred_train = model.predict(X_train).ravel()

### ROC curves and Area Under the Curve (AUC)

In [ ]:
from sklearn.metrics import roc_auc_score # for binary classification if x > 0.5 -> 1 else -> 0
from sklearn.utils import class_weight # to set class_weight="balanced"

In [ ]:
from sklearn.metrics import roc_curve
fpr,tpr,_ = roc_curve(y_true=y_test, y_score=y_pred_test,sample_weight=weights_test)
plt.plot(fpr, tpr, color='blue',lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')

In [ ]:
auc_test = roc_auc_score(y_true=y_test, y_score=y_pred_test,sample_weight=weights_test)
auc_train = roc_auc_score(y_true=y_train.values, y_score=y_pred_train,sample_weight=weights_train)
print("auc test:",auc_test)
print ("auc train:",auc_train)

## Significance Function

$\mathrm{med}[Z_0|1] = \sqrt{q_{0,A}} = \sqrt{2+((s+b)\ln(1+s/b)-s)}$

**asimov significance [arXiv:1007.1727](https://arxiv.org/pdf/1007.1727.pdf) [Eq. 97]**

Likelihood-based statistical test for significance. Need to esimate your sensitivity to MC. Running a toy MC thousands of times, should converge to 'truth'. Asimov is representative of number of sigmas in the gaus case.

Essentially: For an observed number of signal events $s$, what is the significance $Z_0$ with which we would reject the $s = 0$ hypothesis

In [ ]:
from math import sqrt
from math import log
def amsasimov(s,b):
        if b<=0 or s<=0:
            return 0
        try:
            return sqrt(2*((s+b)*log(1+float(s)/b)-s))
        except ValueError:
            print(1+float(s)/b)
            print (2*((s+b)*log(1+float(s)/b)-s))
        #return s/sqrt(s+b)

In [ ]:
#from extra_functions import amsasimov

In [ ]:
int_pred_test_sig = [weights_test[(y_test ==1) & (y_pred_test > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
int_pred_test_bkg = [weights_test[(y_test ==0) & (y_pred_test > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
vamsasimov = [amsasimov(sumsig,sumbkg) for (sumsig,sumbkg) in zip(int_pred_test_sig,int_pred_test_bkg)]
Z = max(vamsasimov)
print("Z:",Z)

In [ ]:
plt.plot(np.linspace(0,1,num=50),vamsasimov, label='Significance (Z = {})'.format(np.round(Z,decimals=2)))


plt.title("NN Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
plt.savefig("Significance_xgb.pdf")
plt.show()

### Plotting NN Score for Signal and Background

In [ ]:
from extra_functions import compare_train_test
compare_train_test(y_pred_train, y_train, y_pred_test, y_test, 
                   xlabel="NN Score", title="NN", 
                   weights_train=weights_train.values, weights_test=weights_test.values)

## What does overtraining look like?

Recipe:
1. Add More layers
2. Add more nodes per layer
3. Train on less data

In [ ]:
#Crazy Example
N = len(X_train)
n = int(N/1000)
print("Using",n,"/",N, "events")

X_small = X_train[:n]
y_small = y_train[:n]
weights_small = weights_train[:n]

In [ ]:
ot_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_small.shape[1],)), # input layer
    tf.keras.layers.Dense(256, activation='relu'), # 1st hiddden layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
ot_model.compile(loss="binary_crossentropy", optimizer="adam")
starting_time = time.time( )

the_overfit = ot_model.fit(X_small, y_small.values, epochs=25 ,validation_data=(X_val, y_val))
#Here, we are not using any sample weights. 
#This is just an example so we omits weights to show val loss

training_time = time.time( ) - starting_time
print("Training time:",training_time)

In [ ]:
ot_y_pred_test = ot_model.predict(X_test).ravel()
ot_y_pred_train = ot_model.predict(X_small).ravel()

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=False, figsize=(15,5))

axes[0].plot(the_overfit.history['loss'],label="training loss")
axes[0].plot(the_overfit.history['val_loss'],label="validation loss")
axes[0].legend(fontsize=15)

axes[1].set_title("Overtrained NN")
compare_train_test(ot_y_pred_train, y_small, ot_y_pred_test, y_test, 
                   xlabel="NN Score", title="Overtrained NN", 
                   weights_train=weights_small.values, weights_test=weights_test.values)
axes[1].legend(loc="upper center",fontsize = 15)

## Exercises

1.   Improve NN AUC and significance by increasing the number of neurons, and layers, epochs, or by any other techniques (google) 
        - (beware of training time)
        - Explore!
        - Loss: [BCE, MSE,]
        - Activations: [relu, leakyrelu, selu, tanh]
2.   Draw NN score for signal and background, training and testing (see overtraining example, or BDT Notebook)
        - feel free to look into *extra_functions.py*
3.   Add Features more features and engineer aditional Features        
___
4.   Draw plot looking at significance for NN ( see BDT notebook), for the same features as BDT
5.   Enable feature permutation importance (see BDT notebook)
6.   Calculate the permutation importance with respect to significance and accuracy. See how Asimov significance changes as features shuffle.






In [ ]:
#Some help for BDT comparisons:
y_pred_xgb = np.load("y_pred_xgb.npy")
y_pred_train_xgb = np.load("./y_pred_train_xgb.npy")
weights_train_xgb = np.load("./weights_train_xgb.npy")
y_test_xgb = np.load("./y_test_xgb.npy")
weights_test_xgb = np.load("./weights_test_xgb.npy")

from xgboost import XGBClassifier
import xgboost
xgb = xgboost.XGBClassifier({'nthread': 4})  # init model
xgb.load_model('xgb.model')  # load data

#The model and predictions from the BDT notebook are now in memory!
#plt.bar(data.columns.values, xgb.feature_importances_)

### Extra: Unweighted Score Plot

In [ ]:
compare_train_test(y_pred_train, y_train, y_pred_test, y_test, 
                   xlabel="NN Score", title="NN", 
                   weights_train=weights_train.values,
                   weights_test=weights_test.values,density=False)